In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio
from PIL import Image
import os
import geopandas as gpd
from shapely.geometry import Point

def initialize_data():
    grid_cells_gdf = gpd.read_file('./data/GridCellsShapefile/GridCells.shp')
    grid_cells_crs = grid_cells_gdf.crs
    return grid_cells_gdf, grid_cells_crs

def retrieve_crossmodels_within_radius(lat, lon, grid_cells_gdf, grid_cells_crs):
    '''
    Retrieves all Crossmodel indices within a specified radius of a given latitude and longitude.
    
    Parameters:
    - lat: Latitude of the location.
    - lon: Longitude of the location.
    - radius_km: The radius in kilometers around the point to retrieve Crossmodel indices.
    - grid_cells_gdf: GeoDataFrame of the grid cells.
    - grid_cells_crs: Coordinate Reference System (CRS) of the grid cells.
    
    Returns:
    - A list containing the Crossmodel indices for the grid cells within the specified radius.
    '''
    # Convert the radius in kilometers to meters (as most CRS use meters)
    radius_meters = 36 * 1000

    # Create a point from the given latitude and longitude
    point = Point(lon, lat)
    point_gseries = gpd.GeoSeries([point], crs="EPSG:4326")  # Assume input is in WGS84

    # Transform the point to match the grid cell CRS
    point_transformed = point_gseries.to_crs(grid_cells_crs)

    # Create a buffer around the point in the correct CRS
    buffer = point_transformed.buffer(radius_meters)
    buffer = buffer.to_crs(grid_cells_crs)

    # Find grid cells that intersect the buffer area
    intersecting_cells = grid_cells_gdf[grid_cells_gdf.intersects(buffer.geometry[0])]

    # Retrieve the Crossmodel indices from the intersecting cells
    crossmodel_indices = intersecting_cells['Crossmodel'].tolist()

    with open('./chat_history/crossmodels.txt', 'a') as f:
        f.write(f"Crossmodels within a 36 km radius of location (lat: {lat}, lon: {lon}):\n")
        for item in crossmodel_indices:
            f.write("%s\n" % item)
    
    return intersecting_cells

grid_cells_gdf, grid_cells_crs = initialize_data()

In [14]:
lat = 35.5942
lon = -105.2228
cross_model = retrieve_crossmodels_within_radius(lat, lon, grid_cells_gdf, grid_cells_crs)
df = pd.read_csv('./data/CCSM_2004_1995_crossmodel.csv', usecols=cross_model['Crossmodel'])

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [3]:
import contextily as ctx
import matplotlib.pyplot as plt
import os
import imageio
from PIL import Image
import numpy as np

# Clip negative values
df = df.clip(lower=0)

In [8]:
monthly_averages = []
num_years = 10
days_per_year = 365
total_days = num_years * days_per_year
days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# Step 3: Compute the monthly average for each month in each year
for year in range(num_years):
    start_year_idx = year * days_per_year
    for days in days_in_month:
        end_idx = start_year_idx + days
        monthly_avg = df.iloc[start_year_idx:end_idx].mean(axis=0)
        monthly_averages.append(monthly_avg)
        start_year_idx = end_idx

# Step 4: Convert the result to a DataFrame
monthly_avg_df = pd.DataFrame(monthly_averages, columns=df.columns)

# Now monthly_avg_df contains the monthly average for each crossmodel for each year
print(monthly_avg_df)

      R108C303   R108C304   R108C305   R109C302   R109C303   R109C304  \
0     6.211290   8.297419   8.939355   3.461935   5.701290   8.858710   
1     2.885357   3.223214   3.580714   2.100000   2.681429   2.969643   
2     9.554839   9.980645   9.995484   8.457097  10.286774  10.313871   
3    23.319333  24.065000  23.103333  18.438333  23.194000  21.540667   
4    23.217419  27.696452  28.953871  21.565806  24.246774  27.304839   
..         ...        ...        ...        ...        ...        ...   
115  12.555806  12.881935  12.338710  12.059032  12.446129  11.788387   
116  18.094333  17.902333  17.845000  15.917667  18.786333  19.179333   
117  24.910323  25.514516  25.272903  23.042903  24.710645  24.641935   
118  19.713667  21.486667  22.558667  17.347000  20.609000  23.020667   
119   0.982903   0.777419   0.592581   1.079032   1.029677   0.675484   

      R109C305   R109C306   R110C301   R110C302  ...   R111C306   R112C302  \
0     8.190645   7.216129   0.820645   1.4203

In [12]:
# function that maps a number to month

def month_map(x):
    x = int(x)%12
    return {
        0: 'Jan',
        1: 'Feb',
        2: 'Mar',
        3: 'Apr',
        4: 'May',
        5: 'Jun',
        6: 'Jul',
        7: 'Aug',
        8: 'Sep',
        9: 'Oct',
        10: 'Nov',
        11: 'Dec'
    }[x]

def generate_images_for_each_timestep(df, grid_cells_gdf, image_dir, figsize=(10, 8)):
    filenames = []
    vmin = 0
    vmax = df.max().max()
    
    for i in range(df.shape[0]):  # Assuming each row is a time step
        grid_cells_gdf['value'] = df.iloc[i].values
        
        fig, ax = plt.subplots(1, 1, figsize=figsize)
        grid_cells_gdf.plot(column='value', cmap='viridis', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True, vmin=vmin, vmax=vmax)
        ctx.add_basemap(ax, crs=grid_cells_gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
        plt.title(f'Year {np.floor(i / 12) + 1995} Month {month_map(i)}')

        filename = os.path.join(image_dir, f'timestep_{i+1}.png')
        plt.savefig(filename, bbox_inches='tight', pad_inches=0)
        filenames.append(filename)
        plt.close()
    return filenames
# Create a directory to save the images
image_dir = 'temp_images'
os.makedirs(image_dir, exist_ok=True)

# Generate images for each time step with spatial mapping
filenames = generate_images_for_each_timestep(monthly_avg_df, cross_model, image_dir)

# Ensure all images are the same size
def resize_image_to_standard_size(image_path, size):
    with Image.open(image_path) as img:
        img = img.resize(size, Image.ANTIALIAS)
        img.save(image_path)

# Get the size of the first image
with Image.open(filenames[0]) as img:
    standard_size = img.size


/Users/yangxinyuxie/Library/Caches/pypoetry/virtualenvs/src-S31xBiIA-py3.11/lib/python3.11/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/yangxinyuxie/Library/Caches/pypoetry/virtualenvs/src-S31xBiIA-py3.11/lib/python3.11/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/yangxinyuxie/Library/Caches/pypoetry/virtualenvs/src-S31xBiIA-py3.11/lib/py

In [1]:
import geopandas as gpd

perimeters = gpd.read_file('./data/Historic_Geomac_Perimeters_Combined/US_HIST_FIRE_PERIMTRS_2000_2018_DD83.shp')


In [2]:
import folium

m5 = folium.Map(location=[35, -100], zoom_start=8)

In [21]:
def initialize_data():
    grid_cells_gdf = gpd.read_file('./data/GridCellsShapefile/GridCells.shp')
    return grid_cells_gdf

In [22]:
grid_cells_gdf = initialize_data()

In [30]:
grid_cells_gdf

,OBJECTID,Crossmodel,Shape_Leng,Shape_Area,geometry
0,1,R161C438,63614.764866,2.529273e+08,"POLYGON ((-9530601.177 4726046.614, -9534793.8..."
1,2,R125C222,61384.219597,2.355013e+08,"POLYGON ((-12959076.287 4395610.472, -12974301..."
2,3,R121C235,61111.892875,2.334164e+08,"POLYGON ((-12754805.395 4355815.951, -12770000..."
3,4,R169C431,64716.234995,2.617618e+08,"POLYGON ((-9605729.481 4879238.815, -9609863.1..."
4,5,R146C497,60142.919468,2.260731e+08,"POLYGON ((-8733007.764 4224658.634, -8738250.3..."
...,...,...,...,...,...
62829,62830,R055C359,54822.101620,1.878414e+08,"POLYGON ((-10965528.715 3400674.224, -10966978..."
62830,62831,R072C387,55964.448729,1.957512e+08,"POLYGON ((-10550370.700 3584259.218, -10552496..."
62831,62832,R085C337,57646.273207,2.076932e+08,"POLYGON ((-11249641.912 3850046.022, -11235259..."
62832,62833,R082C288,57528.265213,2.068438e+08,"POLYGON ((-11942487.554 3816894.598, -11956857..."


In [38]:
from shapely.geometry import Point
radius_meters = 36 * 1000

# Create a point from the given latitude and longitude
point = Point(-122.2711, 37.8044)
point_gseries = gpd.GeoSeries([point], crs="EPSG:4326")  # Assume input is in WGS84

# Transform the point to match the grid cell CRS
point_transformed = point_gseries.to_crs(grid_cells_gdf.crs)

# Create a buffer around the point in the correct CRS
buffer = point_transformed.buffer(radius_meters)
buffer = buffer.to_crs(grid_cells_gdf.crs)

In [39]:
buffer

0    POLYGON ((-13575156.591 4551830.824, -13575329...
dtype: geometry

In [43]:
perimeters = perimeters.to_crs(buffer.crs)
perimeters = perimeters[perimeters.intersects(buffer.geometry[0])]

In [44]:
perimeters

,agency,comments,mapmethod,datecurren,uniquefire,fireyear,incidentna,pooownerun,perimeterd,gisacres,...,firecode,complexpar,poorespons,state,inciwebid,localincid,irwinid,incomplex,complexfir,geometry
9109,State agency,No entry in FireCode; GPS ground,GPS-Walked/ Driven,2011-01-11,2008-CA-005708,2008,QUARRY,None,2008-06-22,210.605784,...,None,None,CA,CA,None,005708,None,N,None,"POLYGON ((-13627331.173 4535135.458, -13627405..."
12208,CDF,No entry in FireCode,Unknown,1899-12-30,2000-CA-000000,2000,Hayward Assist,None,2000-12-31,75.462339,...,None,None,None,CA,None,000000,None,N,None,"POLYGON ((-13586690.251 4530601.167, -13586752..."
14926,NPS,GPS,GPS-Unknown Travel Method,2011-01-13,2009-CAGNP-009008,2009,HAWK,None,2009-09-18,0.370102,...,E7ER,None,CAGNP,CA,None,009008,None,N,None,"POLYGON ((-13635902.212 4554817.359, -13635903..."
15140,Other,GPS ground,GPS-Walked/ Driven,2011-01-11,2008-CA-082428,2008,ANGEL ISLAND FIRE,None,2008-10-13,302.990450,...,None,None,CA,CA,None,082428,None,N,None,"POLYGON ((-13628566.273 4560367.062, -13628591..."
15272,NPS,No entry in FireCode,Unknown,1899-12-30,2000-CA-000035,2000,Pt Bonita,None,2000-10-22,11.039442,...,None,None,None,CA,None,000035,None,N,None,"POLYGON ((-13639749.894 4554413.395, -13639748..."
15425,Unknown,None,Unknown,2010-09-22,2002-CA,2002,PGandE_4,None,2002-08-25,200.154793,...,None,None,None,CA,None,None,None,N,None,"POLYGON ((-13602270.627 4575199.992, -13602297..."
17642,CDF,None,GPS-Walked/ Driven,2010-11-03,2005-CA,2005,BNSF,None,2005-06-13,109.912643,...,None,None,CA,CA,None,005-CA,None,N,None,"POLYGON ((-13604151.668 4581461.640, -13604158..."
22152,Local,None,GPS-Walked/Driven,2010-10-04,2004-CAMRN-000803,2004,Tam,None,2004-05-09,11.791250,...,A26V,None,CAMRN,CA,None,000803,None,N,None,"MULTIPOLYGON (((-13642142.585 4563028.865, -13..."
22462,CDF,None,GPS-Walked/Driven,2010-10-04,2004-CATCU-005048,2004,Highway,None,2004-06-25,624.936477,...,None,None,CATCU,CA,None,005048,None,N,None,"POLYGON ((-13596880.846 4581607.603, -13596916..."
22494,Unknown,None,Unknown,2010-09-27,2003-CA,2003,Mcewen,None,2003-08-12,100.389011,...,None,None,CA,CA,None,None,None,N,None,"POLYGON ((-13600507.589 4581723.566, -13600529..."


In [45]:
m5.add_child(
    folium.features.GeoJson(perimeters)
)

In [46]:
m5